# In this notebook we train sub-linear complexity stochastic GMM algorithms for various datasets

In [ ]:
from pathlib import Path
import time
import shutil
from utils import create_histograms, create_soft_features, read_parameters
import pickle
import subprocess
import numpy as np
import matplotlib.pyplot as plt
from statistics import mean 
from time import gmtime, strftime

## Parameters

In [ ]:
name                    = 'pokerdvs'

# data parameters
training_path           = '../data-generation/exp/'+name+'_train_sp10' # path to the training npy file
test_path               = '../data-generation/exp/'+name+'_test_sp10' # path to the test npy file
sample_percentage       = 100 # take a data sample for the training set instead of full data (between 1 and 100)

# model parameters
algorithm               = 3 # 0:k-means | 1:S-GMM | 2:S-GMM-Prior | 3:D-GMM | 4:D-GMM-Prior
M                       = 500 # number of cluster centers
H                       = 5 # number of clusters considered for each data point
R                       = 40 # number of new samples
Nprime                  = 2**15 # size of subset | set to 0 to disable coreset creation
stream                  = False # stream data from harddrive to build coreset | training data not loaded and only works when using coresets
chain_length            = 10 # chain length for AFK-MC² seeding
convergence_threshold   = 0.0001  # < 1 for convergence threshold | >= 1 for epochs

# output parameters
trials                  = 5 # if > 1 uses intel TBB for parallelisation
top_k                   = 1 # save the k best clusters during inference
inference               = True # save cluster assignments for training and test datasets (when streaming doesn't work for training set)
save_additional_info    = 0 # 1: save centers, error, and priors across iterations | 2: only priors
verbose                 = 0 # display more output (1 or 2 for even more information)

# notebook parameters
clustering_analysis     = False # runs clustering analysis on the results
classification          = True # runs classification on the results
save_figures            = False # save figures after plotting

# classification parameters
torch_mlp               = False # do an MLP classifier instead of logistic regression

## Run C++ GMM standalone executable

In [ ]:
start_time = time.time()
    
algo_type = str(algorithm)

# defining save path
save_path = Path('milled_nbs/{0}/Hp-{1}_R-{2}_M-{3}_mc-{4}_th-{5}/{6}/{7}'.format(name, H, R, M, chain_length, convergence_threshold, algo_type, Nprime))
    
# make folder and if exists empty it  
if save_path.exists():
    shutil.rmtree(save_path)
    
save_path.mkdir(parents=True, exist_ok=True)

if algorithm == 0:
    # k-means baselines
    cmd_list = ['../build/release/km', 
                training_path, 
                test_path, 
                str(save_path), 
                str(sample_percentage),
                str(M), 
                str(Nprime),
                str(int(stream)), 
                str(convergence_threshold), 
                str(trials), 
                str(int(inference)),
                str(int(save_additional_info)), 
                str(int(verbose)), 
               ]
else:
    # gmm algorithms
    cmd_list = ['../build/release/gmm',
                training_path, 
                test_path, 
                str(save_path),
                str(sample_percentage),
                str(algorithm),
                str(M),   
                str(H),                                   
                str(R),                                    
                str(Nprime),
                str(int(stream)),                        
                str(chain_length),                                     
                str(convergence_threshold), 
                str(trials),
                str(top_k),
                str(int(inference)),
                str(0),
                str(int(save_additional_info)), 
                str(int(verbose)),
               ]

# running C++ executable within Python
p = subprocess.run(cmd_list, capture_output=True, universal_newlines=True)
if len(p.stdout) > 0: print(p.stdout)
if len(p.stderr) > 0: print(p.stderr)

print('--- %s seconds ---' % (time.time() - start_time))

## Clustering analysis

In [ ]:
import seaborn as sns
import matplotlib.colors as mcolors

# name of algorithms
algorithms = {'0':'k-means',
              '1':'u-S-GMM',
              '2':'S-GMM'}

# make analysis folder where we dump our results
analysis_path = save_path/'analysis'
analysis_path.mkdir(parents=True, exist_ok=True)

# condition to check if a test dataset is provided
test = False if len(test_path) == 0 else True

# directory where files are saved
save_dir = [save_path] if trials == 1 else [folder for folder in save_path.iterdir() if not folder.name.startswith('.') and folder != analysis_path]

if clustering_analysis:
    start_time = time.time()
    
    print('clustering analysis for algorithm: %s' % algorithms[algo_type], 'over %s run(s)' % len(save_dir))
    
    analysis = {}
    for i, s in enumerate(save_dir):
        print(s)
        folder = s.parts[-2] if trials == 1 else s.parts[-3]

        # read the parameters of the model
        model_parameters = read_parameters(s)
        
        # initialise the key for the algorithm being used if not already done
        if folder not in analysis.keys():
            if folder == '0':
                analysis[folder] = {'M':M,
                                    'lwcs':Nprime, 
                                    'thres':convergence_threshold,
                                    'trials':trials,
                                    'error':np.zeros(trials), 
                                    'dist_eval':np.zeros(trials), 
                                    't_em':np.zeros(trials),
                                    't_seed':np.zeros(trials),
                                    'iter':np.zeros(trials)
                                   }
            else:
                analysis[folder] = {'M':M,
                                    'H':H,
                                    'R':R, 
                                    'lwcs':Nprime, 
                                    'chain':chain_length,
                                    'thres':convergence_threshold,
                                    'trials':trials,
                                    'error':np.zeros(trials), 
                                    'dist_eval':np.zeros(trials), 
                                    't_em':np.zeros(trials),
                                    't_seed':np.zeros(trials),
                                    'iter':np.zeros(trials)
                                   }

        # quantization error
        if test:
            error = np.load(s/'te_quantization.npy')
        else:
            error = np.load(s/'tr_quantization'/str(str(model_parameters.iteration[-1])+'.npy'))
        analysis[folder]['error'][i] = error[0]
        print('error:', error[0])
        
        # distance evaluations
        distances = np.load(s/'distance_evals.npy')
        analysis[folder]['dist_eval'][i] = distances[0]*distances[1]
        print('distance evaluations:', distances[0]*distances[1])
        
        # coreset and seeding runtime
        runtime = np.load(s/'runtime.npy')
        analysis[folder]['t_seed'][i] = runtime[0]
        print('seeding runtime', runtime[0])
        
        # EM runtime
        analysis[folder]['t_em'][i] = runtime[1]
        print('EM runtime:', runtime[1])
        
        # number of iterations
        analysis[folder]['iter'][i] = model_parameters.iteration[-1]
        print('iterations:', model_parameters.iteration[-1])
        
        # Quantization error across iterations
        if save_additional_info > 0:
            error = [np.load(s/'tr_quantization'/str(str(i)+'.npy'))[0] for i in model_parameters.iteration]
            
            error_fig = plt.figure()
            plt.plot(model_parameters.iteration[1:], error[1:])
            plt.title('Quantization error across iterations')
            plt.show()
            
            if save_figures:
                error_fig.savefig(save_path/'analysis'/str(str(i)+'_'+'quantization_iters.pdf'), bbox_inches='tight')
                
        # Free energy across iterations
        free_energy_fig = plt.figure()
        plt.plot(model_parameters.iteration[1:], model_parameters.free_energy[1:])
        plt.title('Free energy across iterations')
        plt.show()
        
        if save_figures:
            free_energy_fig.savefig(save_path/'analysis'/str(str(i)+'_'+'free_energy_iters.pdf'), bbox_inches='tight')
            
        # Sigma across iterations
        sigma_fig = plt.figure()
        plt.plot(model_parameters.iteration[1:], model_parameters.sigma[1:])
        plt.title('Sigma across iterations')
        plt.show()
        
        if save_figures:
            sigma_fig.savefig(save_path/'analysis'/str(str(i)+'_'+'sigma_iters.pdf'), bbox_inches='tight')
        
        # Prior decay
        prior_path = s/'priors'
        if prior_path.exists() and save_additional_info > 0:
            # loading priors
            priors = np.array([np.load(prior_path/str(str(i)+'.npy')) for i in model_parameters.iteration]).T

            # offset according to uniform prior values (we want to see how it moves from the uniform)
            offset = mcolors.TwoSlopeNorm(vcenter=priors[0,0])

            # plotting and formatting
            fig = plt.figure()
            ax = sns.heatmap(offset(priors),cmap=sns.diverging_palette(220, 20, n=128))
            ax.set_title('Priors Decay (M=%s)' % M)
            ax.set_ylabel('Priors')
            ax.set_xlabel('Iterations')
            
            if save_figures:
                fig.savefig(save_path/str('priors_%s.pdf' %M))
                
    # average statistics
    if trials > 1:
        # average iterations
        print('average iterations:', np.mean(analysis[algo_type]['iter']), '\u00b1', np.std(analysis[algo_type]['iter']))
        
        #average runtime
        print('average runtime:',\
              np.mean(analysis[algo_type]['t_seed'] + analysis[algo_type]['t_em']),\
              '\u00b1',\
              np.std(analysis[algo_type]['t_seed'] + analysis[algo_type]['t_em']))
        
        # average operations
        print('average operations:', np.mean(analysis[algo_type]['dist_eval']), '\u00b1', np.std(analysis[algo_type]['dist_eval']))
        
        #average error
        print('average error:', np.mean(analysis[algo_type]['error']), '\u00b1', np.std(analysis[algo_type]['error']))

    # save analysis dictionary    
    filename = save_path/'analysis'/'analysis.pickle'
    with open(filename, 'wb') as f:
        pickle.dump(analysis, f)
            
    print('--- %s seconds ---' % (time.time() - start_time))

## pytorch classification

In [ ]:
if classification and test:
    from logreg import LogisticRegression
    from mlp import MLP
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import TensorDataset, DataLoader
    from torch.optim.lr_scheduler import StepLR
    from sklearn import preprocessing
    
    start_time = time.time()
    
    print('classification for algorithm: %s' % algorithms[algo_type], 'over %s run(s)' % len(save_dir))
    
    scores = np.zeros(len(save_dir))
    for i, s in enumerate(save_dir):
        
        # creating histograms from hard clusters
        train_features, train_labels = create_histograms(s, training_path, M, train=True)
        test_features, test_labels = create_histograms(s, training_path, M, train=False)
        
        # scale features to 0 mean and 1 variance
        scaler = preprocessing.StandardScaler().fit(train_features)
        train_features = scaler.transform(train_features)
        test_features = scaler.transform(test_features)
            
        # creating dataloaders
        training_dataset = TensorDataset(torch.Tensor(train_features),torch.Tensor(train_labels))
        training_dataloader = DataLoader(training_dataset, batch_size=128)
        
        test_dataset = TensorDataset(torch.Tensor(test_features),torch.Tensor(test_labels))
        test_dataloader = DataLoader(test_dataset, batch_size=128)
        
        # finding unique classes
        classes = np.unique(test_labels)
        
        # training classifier
        if torch_mlp:
            device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
            mlp = MLP(train_features.shape[1], 128, len(classes)) # single layer MLP
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.SGD(mlp.parameters(), lr=0.1, momentum=0, weight_decay=0)
            scheduler = StepLR(optimizer, step_size=50, gamma=1)
            
            # train model
            mlp.train()
            for epoch in range(200):  # loop over the dataset multiple times
                for inputs, targets in training_dataloader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    
                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward + backward + optimize
                    outputs = mlp(inputs)
                    loss = criterion(outputs, targets.long())
                    loss.backward()
                    optimizer.step()
            
            # Decay Learning Rate
            scheduler.step()
            
            # test model
            mlp.eval()
            total = 0
            correct = 0
            with torch.no_grad():
                for t_inputs, t_targets in test_dataloader:
                    t_inputs, t_targets = t_inputs.to(device), t_targets.to(device)
                    t_outputs = mlp(t_inputs)
                    t_loss = criterion(t_outputs, t_targets.long())
                    _, predicted = torch.max(t_outputs, 1)
                    total += t_targets.size(0)
                    correct += (predicted == t_targets).sum().item()
            scores[i] = correct / total
        else:
            logreg = LogisticRegression(train_features.shape[1], len(classes), epochs=200, lr=0.01, step_size=30, gamma=1, momentum=0, weight_decay=0)
            logreg.fit(training_dataloader)
            scores[i] = logreg.score(test_dataloader)
        
    # print mean score
    classifier_name = "MLP" if torch_mlp else "Logistic regression"
    print(classifier_name, 'classifier: %s%% \u00B1 %s' % (np.mean(scores[:]*100),np.std(scores[:]*100)))
    
    # save results
    filename = save_path/'analysis'/'classification.npy'
    np.save(filename, scores)
        
    print('--- %s seconds ---' % (time.time() - start_time))